# Imports

**We first import the necessary libraries and define the preprocess_data function. This function takes the CIFAR-10 dataset (X, Y) and preprocesses it, including resizing the images and converting the labels to one-hot encoding.**

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Lambda, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical


ModuleNotFoundError: No module named 'tensorflow'

# Preprocessing Data
- Takes the CIFAR-10 dataset (X, Y) and preprocesses it, resizes the images and converts the labels to one-hot vectors.
- The first line of code creates a new array X_p with the same shape and content as X, but with the data type converted to float32.
- Next, we normalize image data by dividing each pixel value by 255. This scales the pixel values to the range [0, 1].
- Next, we resize images to 224x224 pixels (the required size for ResNet50 model). Convert back to numpy array.
- Convert labels into one-hot vectors using keras 'to_categorical' function. In this case, each label is represented as a binary vector of length 10 (the number of classes in CIFAR-10), with a 1 at the index corresponding to the class and 0s elsewhere.
- Finally, the function returns the preprocessed image data (X_p) and labels (Y_p) as a tuple.

**In summary, the preprocess_data function converts the image data to float32, normalizes it, resizes the images, and converts the labels to one-hot encoding. These preprocessing steps are crucial for preparing the dataset for training a CNN model, such as the one used in this task.**

In [3]:
def preprocess_data(X, Y):
    """preprocesses the data for your model"""
    X_p = X.astype('float32')
    X_p /= 255
    X_p = tf.image.resize(X_p, (224, 224)).numpy()

    Y_p = to_categorical(Y, 10)

    return X_p, Y_p

# Create Model
- Creates a CNN model based on pre-trained ResNet50 architecture with custom top layers for classifying images from CIFAR-10 dataset.
- The first line of code creates a ResNet50 model with weights pre-trained on ImageNet. The include_top argument is set to False, which means that the top layers of the model are not included. This is because we want to add our own custom top layers to the model.
- Next, we iterate through layers of the base model and set their 'trainable' attribute to 'False' freezing layers (their weights won't be updated during training process) allowing us to leverage knowledge learned by the pre-trained model and speeds up process.
- Add lambda layer to resize images using TensorFlow's tf.image.resize function
- Create new top for the model by adding custom layers that will be responsible for the classification task. First, we add a GlobalAveragePooling2D layer, which computes the average value of each feature map from the base model's output. This layer reduces the spatial dimensions and flattens the feature maps.
- Next, we add 'Dense' layer with 10 units (for number of classes in CIFAR-10 dataset) and 'softmax' activation function.
- Create new model combining ResNet50 architecture with custom top layers.
- Finally, compile the model using 'categorical_crossentropy' loss function, 'adam' optimizer, and 'accuracy' metric.

**In summary, the create_model function creates a CNN model based on the pre-trained ResNet50 architecture with custom top layers for CIFAR-10 classification. The base model layers are frozen, and the model is compiled with the Adam optimizer, categorical crossentropy loss, and accuracy as a metric.**

In [4]:
def create_model():
    """creates the model for training"""
    base_model = ResNet50(weights='imagenet', input_shape=(224,224, 3), include_top=False)

    for layer in base_model.layers:
        layer.trainable = False

    resize_layer = Lambda(lambda image: tf.image.resize(image, (224,224)))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    output = Dense(10, activation='softmax')(x)

    # Create new model with our custum top layers
    model = Model(inputs=base_model.input, outputs=output)

    # Compile model
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Main Script

- The CIFAR-10 dataset is loaded using the Keras cifar10.load_data() function, which returns the training and testing data as tuples. The dataset is then preprocessed using the preprocess_data function.
- The create_model function is called to create the CNN model based on the pre-trained ResNet50 architecture, with custom top layers for classifying the CIFAR-10 dataset.
- An ImageDataGenerator is used to augment the training data by applying random transformations, such as rotation, width and height shift, and horizontal flipping. This technique helps improve the model's generalization capabilities.
- The model is then trained for 50 epochs using the augmented training data, with a batch size of 64. The testing data is used as validation data during training.
- The trained model is saved in the current working directory as a file named cifar10.h5.
- The model is evaluated on the testing data using the evaluate method, and the validation accuracy is printed.
- Finally, an assertion checks if the validation accuracy is 87% or higher, as specified in the task requirements.

**In summary, this code snippet loads and preprocesses the CIFAR-10 dataset, creates a CNN model based on the pre-trained ResNet50 architecture, trains the model with data augmentation, saves the model, and evaluates its performance on the testing data. The validation accuracy must be 87% or higher to pass the assertion check.**

In [5]:
if __name__ == '__main__':
    # load and pre-process CIFAR-10 dataset
    (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
    X_train, Y_train = preprocess_data(X_train, Y_train)
    X_test, Y_test = preprocess_data(X_test, Y_test)

    # create model
    model = create_model()

    # train model
    datagen = ImageDataGenerator(
        rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    datagen.fit(X_train)
    history = model.fit(datagen.flow(X_train, Y_train, batch_size=64),
                        epochs=50, validation_data=(X_test, Y_test))

    # save model
    model.save('cifar10.h5')

    # evaluate model
    _, val_acc = model.evaluate(X_test, Y_test, verbose=0)
    print("Validation accuracy: {:.2f}%".format(val_acc * 100))

    # Check if the validation accuracy is 87% or higher
    assert val_acc >= 0.87, "The validation accuracy should be 87% or higher."

NameError: name 'cifar10' is not defined